In [1]:
import random
import chess
import chess.engine
import chess.svg
from IPython.display import SVG, display
import matplotlib.pyplot as plt
from colorama import Fore, Style


print(Fore.RED + "This is red text")
print(Fore.GREEN + "This is green text")
print(Fore.BLUE + "This is blue text")
print(Style.RESET_ALL + "This is normal text")



This is red text
This is green text
This is blue text
This is normal text


In [7]:
from stockfish import Stockfish
stockfish = Stockfish(path="/opt/homebrew/bin/stockfish")

In [2]:
colors = [
    "rgba(0, 255, 0, 1.0)",  
    "rgba(50, 255, 0, 0.8)",  
    "rgba(50, 255, 0, 0.6)",  
    "rgba(50, 255, 0, 0.4)",  
    "rgba(50, 255, 0, 0.2)"   
]


analysis_depth = 5 # How many posible moves will the computer look
com_race = True # # Will the computer be white or black
auto_move = False

In [4]:
def interpret_evaluation(score, turn):
    if score is None:
        return "Unknown"
    # Adjust the score based on whose turn it is
    adjusted_score = score if turn == chess.WHITE else -score
    if adjusted_score > 50:
        return "Excellent"
    elif adjusted_score > 20:
        return "Good"
    elif adjusted_score > -20:
        return "Neutral"
    elif adjusted_score > -50:
        return "Inaccuracy"
    elif adjusted_score > -100:
        return "Mistake"
    else:
        return "Blunder"

In [5]:
def update_evaluation_bar(score):
    plt.clf()
    plt.barh([0], [score], color='green' if score > 0 else 'red')
    plt.xlim(-1000, 1000)
    plt.title('Evaluation Bar')
    plt.xlabel('Centipawns')
    plt.yticks([])
    plt.draw()
    plt.pause(0.001)

# Initialize the evaluation bar
plt.ion()
plt.figure()


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [18]:
def rgb_color(r, g, b):
    return f'\033[38;2;{r};{g};{b}m'


In [45]:
# Reset Game
engine = chess.engine.SimpleEngine.popen_uci("/opt/homebrew/bin/stockfish")
board = chess.Board()


while not board.is_game_over():

    # Use multi-PV mode to get the top analysis_depth moves
    info = engine.analyse(board, chess.engine.Limit(time=1.0), multipv=analysis_depth)
    top_moves = [(entry['pv'][0], entry['score'].relative.score()) for entry in info]

    # Print the top moves
    print("Top moves:", top_moves)

    start_moves = [chess.parse_square(str(moves[0])[:2]) for moves in top_moves]
    end_moves = [chess.parse_square(str(moves[0])[2:]) for moves in top_moves]


   
    # Get the current score of the game
    cur_score = engine.analyse(board, chess.engine.Limit(time=1.0))["score"]

    # Print the moves with color relative to net change in game score
    print(*(f"({rgb_color(-1 * (top_moves[i][1]-cur_score.relative.score()) + 100,(top_moves[i][1]-cur_score.relative.score()) + 100,0)+ str(top_moves[i][0])} {str(top_moves[i][1] - cur_score.relative.score())})" for i in range(analysis_depth)))

    print(Style.RESET_ALL+ str(cur_score))
    print(Style.RESET_ALL+ str(cur_score.relative.score()))

    # Graphics with dynamic arrow color relative to overall game score
    arrows = [chess.svg.Arrow(start_moves[i], end_moves[i], color=f"rgba({-1 * top_moves[i][1]/3 + 100},{top_moves[i][1]/2 + 100},0) ") for i in range(analysis_depth)]
    board_svg = chess.svg.board(board=board, size=200, arrows=arrows)
    display(SVG(board_svg))
            
    # Let the computer or player confirm move       
    if auto_move == True and com_race == True:
        board.push(top_moves[0][0])
        com_race = False
    else: 
        while True:
            my_move = input("Your move: ")
            try:
                move = chess.Move.from_uci(my_move)
                if move in board.legal_moves:
                    board.push(move)                          
                    com_race = True
                    break  # Exit the loop if the move is legal
                else:
                    print(f"{my_move} is an illegal move. Try again.")
            except ValueError:
                print(f"{my_move} is an invalid move format. Try again.")

                if my_move == "q" or my_move == "quit": # kill the game 
                    engine.quit()

        com_race = True

# Close the engine
engine.quit()


Top moves: [(Move.from_uci('e2e4'), 31), (Move.from_uci('g1f3'), 28), (Move.from_uci('d2d4'), 25), (Move.from_uci('e2e3'), 14), (Move.from_uci('c2c3'), 12)]
(e2e4 -5) (g1f3 -8) (d2d4 -11) (e2e3 -22) (c2c3 -24)
PovScore(Cp(+36), WHITE)
36


e4 is an invalid move format. Try again.
Top moves: [(Move.from_uci('e7e6'), -21), (Move.from_uci('e7e5'), -40), (Move.from_uci('c7c5'), -40), (Move.from_uci('c7c6'), -43), (Move.from_uci('d7d6'), -53)]
(e7e6 14) (e7e5 -5) (c7c5 -5) (c7c6 -8) (d7d6 -18)
PovScore(Cp(-35), BLACK)
-35


Top moves: [(Move.from_uci('d2d4'), 35), (Move.from_uci('g1f3'), 29), (Move.from_uci('c2c3'), 28), (Move.from_uci('b1c3'), 26), (Move.from_uci('h2h3'), 15)]
(d2d4 -9) (g1f3 -15) (c2c3 -16) (b1c3 -18) (h2h3 -29)
PovScore(Cp(+44), WHITE)
44


Top moves: [(Move.from_uci('b8c6'), -40), (Move.from_uci('g7g6'), -46), (Move.from_uci('d7d6'), -41), (Move.from_uci('e7e6'), -48), (Move.from_uci('a7a6'), -56)]
(b8c6 -1) (g7g6 -7) (d7d6 -2) (e7e6 -9) (a7a6 -17)
PovScore(Cp(-39), BLACK)
-39


Top moves: [(Move.from_uci('d2d4'), 37), (Move.from_uci('b1c3'), 31), (Move.from_uci('f1e2'), 24), (Move.from_uci('a2a3'), 21), (Move.from_uci('h2h3'), 20)]
(d2d4 -3) (b1c3 -9) (f1e2 -16) (a2a3 -19) (h2h3 -20)
PovScore(Cp(+40), WHITE)
40


...

Top moves: [(Move.from_uci('f6d7'), -92), (Move.from_uci('d6b4'), -100), (Move.from_uci('f6g8'), -162), (Move.from_uci('f6d5'), -212), (Move.from_uci('f6e4'), -276)]
(f6d7 -2) (d6b4 -10) (f6g8 -72) ([38;2;222;-22;0mf6d5 -122) ([38;2;286;-86;0mf6e4 -186)
PovScore(Cp(-90), BLACK)
-90


Top moves: [(Move.from_uci('a2a4'), 98), (Move.from_uci('b2b3'), 70), (Move.from_uci('a1b1'), 70), (Move.from_uci('h1h3'), 70), (Move.from_uci('e1d1'), 55)]
(a2a4 6) (b2b3 -22) (a1b1 -22) (h1h3 -22) (e1d1 -37)
PovScore(Cp(+92), WHITE)
92


Top moves: [(Move.from_uci('d6b4'), -89), (Move.from_uci('e8e7'), -108), (Move.from_uci('c8b7'), -110), (Move.from_uci('b8b6'), -114), (Move.from_uci('b8a8'), -120)]
(d6b4 14) (e8e7 -5) (c8b7 -7) (b8b6 -11) (b8a8 -17)
PovScore(Cp(-103), BLACK)
-103


Top moves: [(Move.from_uci('e1d1'), 93), (Move.from_uci('e1f1'), 62), (Move.from_uci('c1d2'), -12), (Move.from_uci('c2c3'), -66)]


IndexError: list index out of range